# Exploration 

A notebook to explore and build the experimental functions in the Tense and Time Spans project.

In [1]:
# import tools
import collections
import pandas as pd
from tf.fabric import Fabric

In [2]:
# initialize and load TF and data

TF = Fabric(modules='hebrew/etcbc4c', silent=True)

api = TF.load('''
                book chapter verse
                g_cons
                function vt kind 
                mother tab
              ''')

api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.01s B book                 from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s B chapter              from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B verse                from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.13s B g_cons               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.07s B function             from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B vt                   from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.03s B kind                 from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.20s B mother               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.02s B tab                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s Feature overview: 103 for nodes; 5 for edges; 1 configs; 7 computed
  4.85s All features loaded/computed - for

## Clause Level: Tense and Time Markers

Before gathering data at the text level for verbal tense, we will look for tendencies at the clause level. It is intended that the information gathered at clause level can be compared with that from the text level, which in turn can help with the question of whether tense selection can really be influenced by a time marker in another clause. In other words, if the same kinds of tendencies that affect the verbal tense at clause level appear at text level, the case for "time spans" is strengthened.

We begin with some very preliminary data-exploration.

### 1. Time Markers in Verbal Clauses

How many different time markers appear in verbal clauses? What are the most common ones? How diverse are they?

In [3]:
# count time markers
time_markers = collections.defaultdict(list)

# count clauses with multiple time markers
multiples = []

# get time markers from clauses
for clause in F.otype.s('clause'):
    
    # skip clause if not verbal
    if F.kind.v(clause) != 'VC':
        continue
        
    times = [time_phrase for time_phrase in L.d(clause, otype='phrase')
                if F.function.v(time_phrase) == 'Time']
    
    # skip clause if there are no time phrases
    if not times:
        continue
        
    # treat multiple time markers separately
    if len(times) > 1:
        multiples.append((clause,times))
        
    # measure and save other time markers
    else:
        
        # isolate time marker
        marker = times[0]
        
        # get plain-text for the time marker
        consonantal_words = tuple(F.g_cons.v(word) for word in L.d(marker, otype='word'))
        marker_text = ' '.join(consonantal_words)
        
        # save marker and clause node
        time_markers[marker_text].append(clause)

        
# make/present counts

# keep count here
initial_counts = collections.Counter()

# count them
for marker, clause_list in time_markers.items():
    initial_counts[marker] += len(clause_list)
    
# various counts  
top_markers = initial_counts.most_common(50)
top_total = sum(count for marker, count in top_markers)
init_counts_total = sum(initial_counts.values())
    
# put counts into pandas dataframe

initial_dataframe = pd.DataFrame(top_markers, columns=('Markers', 'Counts'))

print('Clauses with single marker: ', init_counts_total)
print('Clauses with multiple markers: ', len(multiples))

Clauses with single marker:  3209
Clauses with multiple markers:  111


In [5]:
print('total top 50: ', top_total)
print('\t\t({}% of all...)'.format(round(top_total/init_counts_total, 2) * 100))
print('total types: ', len(initial_counts.keys()))

initial_dataframe

total top 50:  1683
		(52.0% of all...)
total types:  1021


,Markers,Counts
0,B JWM H HW>,198
1,H JWM,160
2,<TH,78
3,B BQR,76
4,L <WLM,76
5,<D H JWM H ZH,58
6,>Z,57
7,CB<T JMJM,56
8,B JWM,53
9,<D <WLM,45


Notes:

1. prepositions denoting position: e.g. <D, B, L, M, >XR, >Z
2. nouns denoting time/event: JWM, <TH, BQR, <WLM, <RB, MXR, LJLH, 
3. words denoting quantity: KL, numbers
4. word denoting extent: <D
5. the role of article/demonstratives in indicating distance: H, ZH, HW>, HM, HJ>
6. words denoting 